In [77]:
import numpy as np
import pandas as pd
import matplotlib as plot

raw_training = pd.read_csv("data/hw1_q3_train_data.csv")
raw_testing = pd.read_csv("data/hw1_q3_test_data.csv")

df_train_x = raw_training.iloc[:, :-1]
df_train_y = raw_training.iloc[:, -1]
df_test_x = raw_testing.iloc[:, :-1]
df_test_y = raw_testing.iloc[:, -1]


#raw_training.head(5)
df_train_y.value_counts()


class
1    237
0    195
Name: count, dtype: int64

In [78]:
def normalize_dataframe(df,normalization_params):
    normalized_df = df.apply(lambda column: column / normalization_params[column.name])
    return normalized_df

normalization_params = df_train_x.max()

##normalizatio z-score
# normalization_params_mean = df_train_x.mean()
# normalization_params_std = df_train_x.std()
# normalization_params = (df_train_x - normalization_params_mean) / normalization_params_std

df_train_x_norm = normalize_dataframe(df_train_x,normalization_params)
df_test_x_norm = normalize_dataframe(df_test_x,normalization_params)

#print(df_train_x_norm)

#print(df_test_x_norm.loc[1])



# 3a.i
I normalized the data from 0 to 1 by dividing each column by it's max (in the training set). This is simple way to normalize but can be effected by outliers. I'll try training with this first.

In [79]:
counts = df_train_y.value_counts()
print(counts)
237/(237+195)

class
1    237
0    195
Name: count, dtype: int64


0.5486111111111112

# 3a.i

Given: 
   1.  The dataset shows a slight imbalance in class distribution (237 to 195). (could go either way but lean toward f1) 
   2.  We are trying to determine if a mushroom is edible, so False Negatives (undetected poisons mushroom) and False Positives (normal  mushrooms labeled as poisons) should be heavy considered. (F1 is best)

   Therefore I would use F1.

# 3b.i


In [80]:

def euclidean_distance(point1, point2):   
    """takes two np.arrays 
    """
    return np.sqrt(np.sum(np.square(point1 - point2)))

def knn(n,point_to_classify, df_train_x, df_train_y, threshold):
    """ returns (class)
    """
    k = int(np.sqrt(n))
    if(k % 2 == 0):
        k+=1
    assert k%2 != 0

    distances = df_train_x.apply(lambda row: euclidean_distance(point_to_classify, row), axis=1)

    result_df = pd.DataFrame({'distances': distances, 'labels': df_train_y})
    sorted_result_df = result_df.sort_values(by='distances',ascending=True)
    knn = sorted_result_df.head(k)
    label_counts = knn['labels'].value_counts()
    number_of_1 = label_counts.get(1, 0)
    number_of_0 = label_counts.get(0, 0)
 #   print(number_of_1,number_of_0)
    if(number_of_1/(number_of_1+number_of_0) >= threshold):
        return 1
    return 0

#point = np.array((871,6,2,7,0.6680444512544724,960,11,0.0273721330556057)) 
#point = df_test_x_norm.loc[1]
#n = len(df_train_x_norm.index)

#test_point_class = knn(n, point,df_train_x_norm,df_train_y)
#print(test_point_class)

In [81]:
n = len(df_train_x_norm.index)
classes_even = df_test_x_norm.apply(lambda row: knn(n, row, df_train_x_norm, df_train_y, 0.5), axis=1)

In [82]:
n = len(df_train_x_norm.index)
classes_uneven = df_test_x_norm.apply(lambda row: knn(n, row, df_train_x_norm, df_train_y, 0.4), axis=1)

In [83]:
def evaluate(test_predict,test_actual):
    TP = 0
    FP = 0 
    FN = 0 
    TN = 0
    n_test =len(test_actual)
    for i in range(0,n_test):
        if (test_predict[i] == 1 and test_actual[i] == 1):
            TP += 1
        elif(test_predict[i] == 0 and test_actual[i] == 1):
            FN += 1
        elif(test_predict[i] == 0 and test_actual[i] == 0):
            TN += 1
        elif(test_predict[i] == 1 and test_actual[i] == 0):
            FP += 1

    accuracy = (TP +TN)/(TP+FP+FN+TN)
    print(f"Accuracy: {accuracy:.4f}")
    recall = (TP)/(TP+FN)
    print(f"Recall: {recall:.4f}")
    precision = (TP)/(TP+FP)
    print(f"Precision: {precision:.4f}")
    f1 = (2*TP)/(2*TP+FP+FN)
    print(f"F1: {f1:.4f}")
    #print(TP,FP)
    #print(FN,TN)

test_predict_even = classes_even.to_list()
test_predict_uneven = classes_uneven.to_list()

test_actual = df_test_y.to_list()
print("even")
evaluate(test_predict_even,test_actual)
print("uneven")
evaluate(test_predict_uneven,test_actual)



even
Accuracy: 0.5864
Recall: 0.6298
Precision: 0.6298
F1: 0.6298
uneven
Accuracy: 0.6049
Recall: 0.8508
Precision: 0.6039
F1: 0.7064


Accuracy: 0.5864

Recall: 0.6298

Precision: 0.6298

F1: 0.6298

In [84]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score


# Assuming X_train, y_train, X_test, and y_test are already defined
y_test = df_test_y
X_test = df_test_x_norm
X_train = df_train_x_norm
y_train = df_train_y
N = X_train.shape[0]

# Calculate k as the square root of N, rounded to the nearest odd number
k = int(np.sqrt(N))
if k % 2 == 0:
    k += 1
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"recall: {recall:.4f}")
print(f"precision: {precision:.4f}")
print(f"f1: {f1:.4f}")



Accuracy: 0.5864
recall: 0.6298
precision: 0.6298
f1: 0.6298
